<a href="https://colab.research.google.com/github/espada105/NewRecipe/blob/main/Llama_3_1_8b_%2B_Unsloth_2x_%2B_CLIP_NewRecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!       # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep


In [5]:
alpaca_prompt = """

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("AdamCodd/recipe-nlg-alpaca", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/2.84k [00:00<?, ?B/s]

recipes-nlg-alpaca.jsonl.gz:   0%|          | 0.00/150M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/501239 [00:00<?, ? examples/s]

Map:   0%|          | 0/501239 [00:00<?, ? examples/s]

### Train the model


In [34]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [35]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 69,634 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,1.232000
2,1.288800
3,1.214000
4,1.173800
5,1.200900
6,1.179300
7,1.184600
8,1.183600
9,1.178900
10,1.124800


Step,Training Loss
1,1.232000
2,1.288800
3,1.214000
4,1.173800
5,1.200900
6,1.179300
7,1.184600
8,1.183600
9,1.178900
10,1.124800


### Inference

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [37]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
from transformers import TextStreamer

# CLIP 모델 및 프로세서 초기화
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 이미지 로드
image_path = "/content/drive/MyDrive/first_training/sample_image.jpg"

# 식재료 리스트 정의
ingredients = [
    "apple", "banana", "orange", "strawberry", "grape", "pear", "peach", "plum", "kiwi", "mango",
    "pineapple", "watermelon", "lemon", "lime", "raspberry", "blueberry", "blackberry", "cherry",
    "onion", "carrot", "tomato", "potato", "broccoli", "lettuce", "eggplant", "garlic", "cucumber",
    "bell pepper", "spinach", "cabbage", "cauliflower", "asparagus", "zucchini", "celery", "kale",
    "mushroom", "pumpkin", "sweet potato", "corn", "green beans", "peas", "radish", "beetroot",
    "basil", "parsley", "cilantro", "mint", "rosemary", "thyme", "sage", "oregano", "ginger",
    "chicken", "beef", "pork", "salmon", "tuna", "shrimp", "tofu", "eggs", "lamb",
    "cheese", "milk", "yogurt", "butter", "cream",
    "almonds", "walnuts", "peanuts", "cashews", "sesame seeds", "chia seeds", "sunflower seeds",
    "rice", "pasta", "bread", "quinoa", "oats", "barley", "wheat", "corn flour",
    "beans", "lentils", "chickpeas", "soybeans", "edamame",
    "bok choy", "napa cabbage", "daikon", "lotus root", "bamboo shoots", "water chestnuts",
    "seaweed", "tofu skin", "shiitake mushroom", "kimchi", "gochugaru", "miso",
    "cinnamon", "cumin", "turmeric", "paprika", "coriander", "cardamom", "nutmeg",
    "saffron", "vanilla", "bay leaves", "star anise", "cloves",
    "tempeh", "seitan", "jackfruit", "mushroom protein", "plant-based meat",
    "cod", "halibut", "crab", "mussels", "oysters", "scallops", "squid", "octopus"
]

def detect_ingredients(image_path, confidence_threshold=0.1):
    """
    CLIP 모델로 이미지에서 식재료 감지
    """
    image = Image.open(image_path)
    # 텍스트와 이미지 유사도 계산
    inputs = processor(text=ingredients, images=image, return_tensors="pt", padding=True)
    clip_outputs = clip_model(**inputs)

    # 유사도 계산
    logits_per_image = clip_outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

    # confidence_threshold 이상의 결과만 필터링
    filtered_probs = probs[0][probs[0] >= confidence_threshold]
    top_k = max(1, len(filtered_probs))
    top_probs, top_labels = probs.topk(top_k, dim=1)

    # 감지된 결과 저장
    results = []
    for i in range(top_k):
        ingredient = ingredients[top_labels[0][i].item()]
        confidence = top_probs[0][i].item()
        results.append({"ingredient": ingredient, "confidence": confidence})

    result_outputs = [item["ingredient"] for item in results]

    return result_outputs

# 식재료 감지
result_output = detect_ingredients(image_path)

# FastLanguageModel 추론 모드로 전환
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# 입력 생성
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "You will generate one cooking recipe. List all necessary ingredients and give detailed steps.",  # instruction
            result_output,  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

# 텍스트 스트리머 초기화
text_streamer = TextStreamer(tokenizer)

# 모델로부터 출력 생성
outputs = model.generate(**inputs, max_new_tokens=1000)

# 출력 처리
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 특수 토큰 제거
response = response.replace("<|begin_of_text|>", "").replace("<|end_of_text|>", "").strip().replace("DIRECTIONS", "RECIPE").strip()

# "Response:" 이후 부분만 가져오기
if "Response:" in response:
    response = response.split("Response:")[-1].strip()

# 레시피 제목 추가
recipe_title = response.split("\n")[0]  # 첫 번째 줄을 레시피 제목으로 사용
ingredients_and_directions = "\n".join(response.split("\n")[1:])  # 나머지 부분

# 최종 출력 형식
final_response = f"""

         *=:    !=-                           *=***=:                      **
         ~==    -=                            =   -=~                     ==
        ~ **.  -!                            =    =*
       ~.-=*. -!    .:.* .**,**~.-*~        =    =,     ,*.*-  -****   **   -***!     ;,*-
      ~. ;=~ -!   :*  =..-=. *. .:         =~~~*:     -*  =~.-*.   ;  *;   !; :=   *=  =~
     ~. .!=,-!   =  !:   *: :* -,         = ;*,     ,=   =. !;    ; *:    !;  "= !;   ~=
    ,~   ;=;!  ;*:-:    *; != ;.         =  =!     ;=:,~. ,=.      ,=-   !*:!~ ,=;,,~
   ,~   -==!  =!        = - ~;.         =   =;    =       =-      -=    !;    ;=
   ~     -=!  =!   =    != -=..         =   -=;   =,   ~. !;      ;*-   !;    *=    ;
  :*     -!  ;!,,!      =  =           =    ~=~..*=,,:.   *-,,:  =-,  ,!!    .=,,~ \n\n\n
I recommend a recipe for the ingredients in your picture!
=====================NewRecipe=====================
Recipe Title: {recipe_title}
{ingredients_and_directions}
"""

# 중복된 응답을 저장할 리스트 초기화
previous_responses = []

# 중복 제거
if final_response not in previous_responses:
    previous_responses.append(final_response)  # 응답을 저장
    print(final_response)
else:
    print("중복된 응답이 생성되었습니다.")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]



         *=:    !=-                           *=***=:                      **
         ~==    -=                            =   -=~                     ==
        ~ **.  -!                            =    =*
       ~.-=*. -!    .:.* .**,**~.-*~        =    =,     ,*.*-  -****   **   -***!     ;,*-
      ~. ;=~ -!   :*  =..-=. *. .:         =~~~*:     -*  =~.-*.   ;  *;   !; :=   *=  =~
     ~. .!=,-!   =  !:   *: :* -,         = ;*,     ,=   =. !;    ; *:    !;  "= !;   ~=
    ,~   ;=;!  ;*:-:    *; != ;.         =  =!     ;=:,~. ,=.      ,=-   !*:!~ ,=;,,~
   ,~   -==!  =!        = - ~;.         =   =;    =       =-      -=    !;    ;=
   ~     -=!  =!   =    != -=..         =   -=;   =,   ~. !;      ;*-   !;    *=    ;
  :*     -!  ;!,,!      =  =           =    ~=~..*=,,:.   *-,,:  =-,  ,!!    .=,,~ 



I recommend a recipe for the ingredients in your picture!
=====================NewRecipe=====================
Recipe Title: [Easy Crock Pot Onion Soup]

[INGREDIENTS]
'onion', 'gar

### Text입력

In [38]:
# FastLanguageModel 추론 모드로 전환
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

def get_response(user_input):
    # 입력 생성
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                user_input,  # instruction (사용자 입력)
                "",  # input (빈 문자열)
                "",  # output (빈 문자열)
            )
        ], return_tensors="pt").to("cuda")

    # 텍스트 스트리머 초기화
    text_streamer = TextStreamer(tokenizer)

    # 모델로부터 출력 생성
    outputs = model.generate(**inputs, max_new_tokens=1000)

    # 출력 처리
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 특수 토큰 제거
    response = response.replace("<|begin_of_text|>", "").replace("<|end_of_text|>", "").strip()

    # "Response:" 이후 부분만 가져오기
    if "Response:" in response:
        response = response.split("Response:")[-1].strip()

    return response

# 대화 루프
while True:
    user_input = input("\n질문을 입력하세요 (종료하려면 'quit' 입력): ")
    if user_input.lower() == 'quit':
        break

    response = get_response(user_input)
    print("\n응답:", response)


질문을 입력하세요 (종료하려면 'quit' 입력): orange

응답: [Orange Marmalade]

[INGREDIENTS]
orange, water, sugar

[DIRECTIONS]:
- Wash and scrub the oranges well
- Cut into 1/4-inch slices
- Place in a large saucepan and cover with water
- Bring to a boil and simmer 30 minutes
- Drain the oranges and save the juice
- Combine the sugar and 3 cups of the orange juice in a large pot
- Bring to a boil, stirring to dissolve the sugar
- Add the oranges and simmer gently for 1 hour
- Stir occasionally to prevent sticking
- Add more orange juice if necessary
- The marmalade is done when a spoonful of it placed in a saucer and cooled forms a skin
- Pour into hot sterilized jars and seal
- Process in boiling water bath for 10 minutes.

질문을 입력하세요 (종료하려면 'quit' 입력): how long take it recipe?

응답: [Quick And Easy Mince Pies]

[INGREDIENTS]
onion, lemon juice, flour, water, vegetable oil, ground black pepper, ground ginger, beef, salt, nutmeg, cinnamon

[DIRECTIONS]:
- Heat oil in a large skillet over medium heat
- 

### 음성으로 입력

#### voice_recoder.py

In [47]:
import pyaudio
import wave
import speech_recognition as sr

def record_audio(filename, duration=5):
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    rate = 16000

    audio = pyaudio.PyAudio()

    stream = audio.open(format=format,
                        channels=channels,
                        rate=rate, input=True,
                        frames_per_buffer=chunk)

    print("녹음 시작...")
    frames = []

    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("녹음 종료.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

def transcribe_audio(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='ko-KR')
            print("인식된 텍스트:", text)
        except sr.UnknownValueError:
            print("음성을 인식할 수 없습니다.")
        except sr.RequestError as e:
            print(f"구글 음성 인식 서비스에 접근할 수 없습니다: {e}")

def start_recording():
    filename = "recorded_audio.wav"
    record_audio(filename, duration=5)
    transcribe_audio(filename)

if __name__ == "__main__":
    start_recording()

()

In [45]:
from google.colab import drive
import os
import io
from google.cloud import speech

# Google Cloud Speech-to-Text API 키 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/first_training/NewRecipeAPIKEY.json"

# wav 파일 경로
audio_file_path = "/content/drive/MyDrive/first_training/녹음.wav"

# 음성 파일 전사 함수
def transcribe_speech(audio_file_path):
    try:
        # Google Speech Client 초기화
        client = speech.SpeechClient()

        # 음성 파일 읽기
        with io.open(audio_file_path, "rb") as audio_file:
            content = audio_file.read()

        # RecognitionAudio 및 RecognitionConfig 설정
        audio = speech.RecognitionAudio(content=content)
        config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,  # wav 파일은 LINEAR16 사용
            sample_rate_hertz=16000,  # wav 파일의 샘플 레이트
            language_code="ko-KR"     # 한국어
        )

        # 음성 인식 요청
        response = client.recognize(config=config, audio=audio)

        # 결과 출력
        for result in response.results:
            print("인식된 텍스트:", result.alternatives[0].transcript)
    except Exception as e:
        print(f"Error during transcription: {e}")

# 음성 전사 실행
transcribe_speech(audio_file_path)


Error during transcription: name 'speech' is not defined


#### voice.py

In [ ]:
import os
from google.cloud import speech
import io

# API KEY
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "NewRecipeAPIKEY.json"

def transcribe_speech(audio_file_path):
    client = speech.SpeechClient()
    with io.open(audio_file_path, "rb") as audio_file:
        content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US"
    )
    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print("Recognized Text:", result.alternatives[0].transcript)

transcribe_speech("your-audio-file.wav")


#### 음성인식으로 레시피 찾기

In [ ]:
import os
import io
from google.cloud import speech
import pyaudio
import wave
from transformers import TextStreamer

# Google Cloud Speech-to-Text API 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "NewRecipeAPIKEY.json"

# FastLanguageModel 추론 모드 활성화
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

def record_audio(filename, duration=5):
    """음성 녹음 및 저장"""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    rate = 16000

    audio = pyaudio.PyAudio()

    stream = audio.open(format=format,
                        channels=channels,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

    print("녹음 시작...")
    frames = []

    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("녹음 종료.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

def transcribe_speech(audio_file_path):
    """Google Cloud Speech-to-Text를 사용하여 음성 파일을 텍스트로 변환"""
    client = speech.SpeechClient()
    with io.open(audio_file_path, "rb") as audio_file:
        content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="ko-KR"
    )
    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        return result.alternatives[0].transcript
    return None

def get_response(user_input):
    """FastLanguageModel로부터 응답 생성"""
    # 입력 생성
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                user_input,  # instruction (음성 텍스트 입력)
                "",  # input (빈 문자열)
                "",  # output (빈 문자열)
            )
        ], return_tensors="pt").to("cuda")

    # 텍스트 스트리머 초기화
    text_streamer = TextStreamer(tokenizer)

    # 모델로부터 출력 생성
    outputs = model.generate(**inputs, max_new_tokens=1000)

    # 출력 처리
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 특수 토큰 제거
    response = response.replace("<|begin_of_text|>", "").replace("<|end_of_text|>", "").strip()

    # "Response:" 이후 부분만 가져오기
    if "Response:" in response:
        response = response.split("Response:")[-1].strip()

    return response

# 대화 루프
while True:
    print("\n음성을 녹음합니다 (종료하려면 'Ctrl+C'를 누르세요)...")
    audio_filename = "recorded_audio.wav"
    record_audio(audio_filename, duration=5)  # 음성 녹음

    print("음성을 텍스트로 변환 중...")
    user_input = transcribe_speech(audio_filename)  # 음성을 텍스트로 변환
    if not user_input:
        print("음성을 인식하지 못했습니다. 다시 시도해주세요.")
        continue

    print("\n인식된 질문:", user_input)
    response = get_response(user_input)
    print("\n응답:", response)


## 성능평가

### ROUGE


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=db60ad6d31270274e7d1ecbb8143e82a008342e1064a4f0ad47b0c49a48c7d75
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from rouge_score import rouge_scorer

# # 미리 정의된 NER와 directions 매핑 테이블
# NER_TO_DIRECTIONS = {
#     "potatoes": ["Peel and dice the potatoes.", "Boil the diced potatoes until soft."],
#     "green onions": ["Chop the green onions finely."],
#     "sugar": ["Mix sugar into the sauce for sweetness."],
#     "bacon": ["Cube the bacon and fry until crispy."],
#     "flour": ["Use flour to thicken the sauce."],
#     "vinegar": ["Add vinegar for a tangy flavor."],
#     "water": ["Add water as needed to adjust consistency."]
# }

# # ROUGE 점수 계산 함수
# def calculate_rouge(reference_text, generated_text):
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
#     return scorer.score(reference_text, generated_text)

# # ROUGE 점수 출력 함수
# def print_rouge_scores(scores):
#     print("\n=== ROUGE Scores ===")
#     print(f"ROUGE-1: Precision: {scores['rouge1'].precision:.4f}, Recall: {scores['rouge1'].recall:.4f}, F1-Score: {scores['rouge1'].fmeasure:.4f}")
#     print(f"ROUGE-2: Precision: {scores['rouge2'].precision:.4f}, Recall: {scores['rouge2'].recall:.4f}, F1-Score: {scores['rouge2'].fmeasure:.4f}")
#     print(f"ROUGE-L: Precision: {scores['rougeL'].precision:.4f}, Recall: {scores['rougeL'].recall:.4f}, F1-Score: {scores['rougeL'].fmeasure:.4f}")

# # NER 기반 directions 생성 함수
# def generate_directions_from_ner(ner_list):
#     directions = []
#     for ingredient in ner_list:
#         if ingredient in NER_TO_DIRECTIONS:
#             directions.extend(NER_TO_DIRECTIONS[ingredient])
#     return directions

# # 샘플 데이터
# sample_data = {
#     "title": "German Potato Salad",
#     "ingredients": [
#         "12 medium cooked potatoes",
#         "1 c. chopped green onions",
#         "1/2 c. sugar",
#         "1/4 lb. bacon",
#         "2 Tbsp. flour",
#         "1/2 c. vinegar",
#         "1/2 c. water"
#     ],
#     "NER": ["potatoes", "green onions", "sugar", "bacon", "flour", "vinegar", "water"]
# }

# # 샘플 입력 텍스트 생성
# sample_input = f"Using ingredients: {', '.join(sample_data['NER'])}. Create a detailed step-by-step recipe."

# # NER에 따른 directions 생성
# dynamic_directions = generate_directions_from_ner(sample_data["NER"])
# reference_text = " ".join(dynamic_directions)

# print("\n=== Generated Directions from NER ===")
# print(reference_text)

# # 모델 출력 생성
# inputs = tokenizer([sample_input], return_tensors="pt").to("cuda")
# outputs = model.generate(**inputs, max_new_tokens=300)
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # ROUGE 점수 계산
# rouge_scores = calculate_rouge(reference_text, generated_text)

# # 출력
# print("\n=== Generated Recipe ===")
# print(generated_text)
# print_rouge_scores(rouge_scores)



=== Generated Directions from NER ===
Peel and dice the potatoes. Boil the diced potatoes until soft. Chop the green onions finely. Mix sugar into the sauce for sweetness. Cube the bacon and fry until crispy. Use flour to thicken the sauce. Add vinegar for a tangy flavor. Add water as needed to adjust consistency.

=== Generated Recipe ===
Using ingredients: potatoes, green onions, sugar, bacon, flour, vinegar, water. Create a detailed step-by-step recipe.

=== ROUGE Scores ===
ROUGE-1: Precision: 0.5882, Recall: 0.2041, F1-Score: 0.3030
ROUGE-2: Precision: 0.0625, Recall: 0.0208, F1-Score: 0.0312
ROUGE-L: Precision: 0.4706, Recall: 0.1633, F1-Score: 0.2424


### BLEU, ROUGE

In [ ]:
!pip install nltk


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# BLEU 점수 계산 함수
def calculate_bleu(reference, candidate):
    smoothie = SmoothingFunction().method4  # 스무딩 적용
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

# ROUGE 점수 계산 함수
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference_text, generated_text)

# ROUGE 점수 출력 함수
def print_rouge_scores(scores):
    print("\n=== ROUGE Scores ===")
    print(f"ROUGE-1: Precision: {scores['rouge1'].precision:.4f}, Recall: {scores['rouge1'].recall:.4f}, F1-Score: {scores['rouge1'].fmeasure:.4f}")
    print(f"ROUGE-2: Precision: {scores['rouge2'].precision:.4f}, Recall: {scores['rouge2'].recall:.4f}, F1-Score: {scores['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L: Precision: {scores['rougeL'].precision:.4f}, Recall: {scores['rougeL'].recall:.4f}, F1-Score: {scores['rougeL'].fmeasure:.4f}")

# 샘플 입력 및 참조 텍스트
# 데이터셋 샘플에 기반한 입력 및 참조 텍스트 생성
sample_data = {
    "title": "German Potato Salad",
    "ingredients": [
        "12 medium cooked potatoes",
        "1 c. chopped green onions",
        "1/2 c. sugar",
        "1/4 lb. bacon",
        "2 Tbsp. flour",
        "1/2 c. vinegar",
        "1/2 c. water"
    ],
    "directions": [
        "Cube bacon and fry crisp.",
        "Add vinegar and water.",
        "Thicken with flour and sugar mixed.",
        "Add potatoes and onions.",
        "Heat slowly.",
        "Salt and pepper to taste."
    ],
    "NER": ["potatoes", "green onions", "sugar", "bacon", "flour", "vinegar", "water"]
}

# 샘플 입력 텍스트 생성
sample_input = f"Using ingredients: {', '.join(sample_data['NER'])}. Create a detailed step-by-step recipe."

# 참조 텍스트 준비
reference_texts = sample_data["directions"]  # 데이터셋의 directions 필드
reference_text = " ".join(reference_texts)  # 참조 텍스트를 하나의 문자열로 병합

# 결과 출력
print("Sample Input:")
print(sample_input)
print("\nReference Text:")
print(reference_text)

# 모델 출력 생성
inputs = tokenizer([sample_input], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
candidate_text = generated_text.split()

# BLEU 점수 계산
bleu_score = calculate_bleu([reference_text.split()], candidate_text)

# ROUGE 점수 계산
rouge_scores = calculate_rouge(reference_text, generated_text)

# 출력
print("\n=== Generated Recipe ===")
print(generated_text)
print("\n=== BLEU Score ===")
print(f"BLEU Score: {bleu_score:.4f}")
print_rouge_scores(rouge_scores)


Sample Input:
Using ingredients: potatoes, green onions, sugar, bacon, flour, vinegar, water. Create a detailed step-by-step recipe.

Reference Text:
Cube bacon and fry crisp. Add vinegar and water. Thicken with flour and sugar mixed. Add potatoes and onions. Heat slowly. Salt and pepper to taste.

=== Generated Recipe ===
Using ingredients: potatoes, green onions, sugar, bacon, flour, vinegar, water. Create a detailed step-by-step recipe.

=== BLEU Score ===
BLEU Score: 0.0080

=== ROUGE Scores ===
ROUGE-1: Precision: 0.4118, Recall: 0.2692, F1-Score: 0.3256
ROUGE-2: Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
ROUGE-L: Precision: 0.1765, Recall: 0.1154, F1-Score: 0.1395


In [ ]:
import random
# 데이터셋 로드
dataset = load_dataset("Jeon3458/newrecipe", split="train")

# BLEU 점수 계산 함수
def calculate_bleu(reference, candidate):
    smoothie = SmoothingFunction().method4  # 스무딩 적용
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

# ROUGE 점수 계산 함수
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference_text, generated_text)


# 랜덤 샘플 추출
random_sample = random.choice(dataset)

# 샘플 데이터 구성
title = random_sample["title"]
ingredients = random_sample["ingredients"]
directions = random_sample["directions"]

# NER 기반 입력 텍스트 생성
input_prompt = f"Using ingredients: {', '.join(ingredients)}. Create a detailed step-by-step recipe."

# 참조 텍스트
reference_text = " ".join(directions)

# 모델로부터 생성된 텍스트
inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=2048)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# BLEU 점수 계산
bleu_score = calculate_bleu([reference_text.split()], generated_text.split())

# ROUGE 점수 계산
rouge_scores = calculate_rouge(reference_text, generated_text)

# 결과 출력
print(f"=== Random Sample Title ===\n{title}")
print(f"\n=== Input Prompt ===\n{input_prompt}")
print(f"\n=== Reference Directions ===\n{reference_text}")
print(f"\n=== Generated Directions ===\n{generated_text}")

print("\n=== BLEU Score ===")
print(f"{bleu_score:.4f}")

print("\n=== ROUGE Scores ===")
print(f"ROUGE-1: Precision: {rouge_scores['rouge1'].precision:.4f}, Recall: {rouge_scores['rouge1'].recall:.4f}, F1-Score: {rouge_scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-2: Precision: {rouge_scores['rouge2'].precision:.4f}, Recall: {rouge_scores['rouge2'].recall:.4f}, F1-Score: {rouge_scores['rouge2'].fmeasure:.4f}")
print(f"ROUGE-L: Precision: {rouge_scores['rougeL'].precision:.4f}, Recall: {rouge_scores['rougeL'].recall:.4f}, F1-Score: {rouge_scores['rougeL'].fmeasure:.4f}")



Repo card metadata block was not found. Setting CardData to empty.


=== Random Sample Title ===
Pizza Dip

=== Input Prompt ===
Using ingredients: [, ", 8,  , o, z, .,  , c, r, e, a, m,  , c, h, e, e, s, e, ", ,,  , ", 1, 5,  , o, z, .,  , c, a, n,  , p, i, z, z, a,  , s, a, u, c, e, ", ,,  , ", 3,  , o, z, .,  , p, e, p, p, e, r, o, n, i, ,,  , d, i, c, e, d, ", ,,  , ", 1, 6,  , o, z, .,  , s, h, r, e, d, d, e, d,  , M, o, z, z, a, r, e, l, l, a,  , c, h, e, e, s, e, ", ]. Create a detailed step-by-step recipe.

=== Reference Directions ===
[ " M a k e s   2   p a n s . " ,   " L a y e r   i n g r e d i e n t s   i n   p i e   p a n s . " ,   " B a k e   o r   m i c r o w a v e   u n t i l   m e l t e d . " ,   " U s e   a s   a   d i p   w i t h   W h e a t   T h i n s . " ]

=== Generated Directions ===
Using ingredients: [, ", 8, , o, z,., , c, r, e, a, m, , c, h, e, e, s, e, ",,, , ", 1, 5, , o, z,., , c, a, n, , p, i, z, z, a, , s, a, u, c, e, ",,, , ", 3, , o, z,., , p, e, p, p, e, r, o, n, i,,, , d, i, c, e, d, ",,, , ", 1, 6, , o, z,., , s, h

### NER로 새롭게 평가

In [ ]:
import random
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# 데이터셋 로드
dataset = load_dataset("Jeon3458/newrecipe", split="train")

# BLEU 점수 계산 함수
def calculate_bleu(reference, candidate):
    smoothie = SmoothingFunction().method4  # 스무딩 적용
    return sentence_bleu([reference.split()], candidate.split(), smoothing_function=smoothie)

# ROUGE 점수 계산 함수
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference_text, generated_text)

# 랜덤 NER 기반 샘플 추출 및 평가
def evaluate_random_ner(dataset, model, tokenizer):
    # 랜덤 샘플 추출
    random_sample = random.choice(dataset)

    # 샘플 데이터 구성
    title = random_sample["title"]
    ingredients = random_sample["ingredients"]
    directions = random_sample["directions"]

    # 랜덤 NER 선택
    random_ner = random.choice(ingredients)

    # 모델 입력 텍스트 생성
    input_prompt = f"Using ingredient: {random_ner}. Create a detailed step-by-step recipe."

    # 참조 텍스트
    reference_text = " ".join(directions)

    # 모델로부터 생성된 텍스트
    inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=300)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # BLEU 점수 계산
    bleu_score = calculate_bleu(reference_text, generated_text)

    # ROUGE 점수 계산
    rouge_scores = calculate_rouge(reference_text, generated_text)

    # 결과 출력
    print(f"=== Random Sample Title ===\n{title}")
    print(f"\n=== Random NER (Ingredient) ===\n{random_ner}")
    print(f"\n=== Input Prompt ===\n{input_prompt}")
    print(f"\n=== Reference Directions ===\n{reference_text}")
    print(f"\n=== Generated Directions ===\n{generated_text}")

    print("\n=== BLEU Score ===")
    print(f"{bleu_score:.4f}")

    print("\n=== ROUGE Scores ===")
    print(f"ROUGE-1: Precision: {rouge_scores['rouge1'].precision:.4f}, Recall: {rouge_scores['rouge1'].recall:.4f}, F1-Score: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f"ROUGE-2: Precision: {rouge_scores['rouge2'].precision:.4f}, Recall: {rouge_scores['rouge2'].recall:.4f}, F1-Score: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L: Precision: {rouge_scores['rougeL'].precision:.4f}, Recall: {rouge_scores['rougeL'].recall:.4f}, F1-Score: {rouge_scores['rougeL'].fmeasure:.4f}")

# 평가 실행
evaluate_random_ner(dataset, model, tokenizer)
